In [1]:
# Adopted from SMRT code: https://github.com/smrt-model/smrt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys
sys.path.append("../smrt")
from smrt import make_snowpack, make_model, make_soil, sensor_list, make_ice_column, PSU
from smrt.atmosphere.simple_isotropic_atmosphere import SimpleIsotropicAtmosphere
from smrt.microstructure_model.sticky_hard_spheres import StickyHardSpheres
import smrt
import argparse

In [2]:
class Emissivity:
    def __init__(self, instr_parameters=None, model_parameters=None):
                
        # Instrumental parameters
        if instr_parameters is None:
            self.fq_list = [37e9]
            self.polarizations = ['h', 'v']
            self.theta = 55
        else:
            self.fq_list = instr_parameters['fq_list']
            self.polarizations = instr_parameters['polarizations']
            self.theta = instr_parameters['theta']
            
        self.radiometer = sensor_list.passive(self.fq_list, self.theta)
        n_max_stream = 128  # TB calculation is more accurate if number of streams is increased (currently: default = 32)
        
        # create the EM Model
        self.m = make_model("iba", "dort",
                            rtsolver_options={"n_max_stream": n_max_stream,
                                              'phase_normalization': "forced"})
        
        # Snow and surface parameters
        self.sn_ms_model_list = ['autocorrelation', 'exponential', 'gaussian_random_field',
                            'homogeneous', 'independent_sphere', 'sampled_autocorrelation',
                            'sticky_hard_spheres','test_autocorrelation', 'test_exponential',
                            'test_sticky_hard_spheres', 'teubner_strey', 'unified_autocorrelation',
                            'unified_scaled_exponential', 'unified_sticky_hard_spheres', 'unified_teubner_strey']
        
        if model_parameters is None:
            self.e_equation = 1
            self.substrate = 'land'
            self.roughness_rms = 0.01
            self.stickiness = 0.2
            self.radius = 0.3e-3
            self.sn_ms_model = self.sn_ms_model_list[6]
            self.sn_thickness = [1]
            self.sn_density = [320]
            self.sn_temp = 265
            self.surface_temp =self.sn_temp
        else:
            self.e_equation = model_parameters['e_equation']
            self.substrate = model_parameters['substrate']
            self.roughness_rms = model_parameters['roughness_rms']
            self.stickiness = model_parameters['stickiness']
            self.radius = model_parameters['radius']
            self.sn_ms_model = self.sn_ms_model_list[model_parameters['sn_ms_model']]
            
            if type(model_parameters['sn_thickness']) == list:
                self.sn_thickness = model_parameters['sn_thickness']
            else:
                self.sn_thickness = [model_parameters['sn_thickness']]
                
            if type(model_parameters['sn_density']) == list:
                self.sn_density = model_parameters['sn_density']
            else:
                self.sn_density = [model_parameters['sn_density']]           

            self.sn_temp = model_parameters['sn_temp']
            self.surface_temp = model_parameters['surface_temp']
            
        print(f'Start initializing snowpack with:')
        print(f'Microstructure model: {self.sn_ms_model}')
        print(f'Snow density: {self.sn_density}')
        print(f'Snow temperature: {self.sn_temp}')
        print(f'Sphere radius: {self.radius}')
        print(f'Stickiness: {self.stickiness}')
        self.snowpack = make_snowpack(microstructure_model=self.sn_ms_model,
                                density=self.sn_density,
                                thickness=self.sn_thickness,
                                temperature=self.sn_temp,
                                radius=self.radius,
                                stickiness=self.stickiness)
        print('Done.\n')
        
        print('Adding sustrate...')
        self.add_substrate()
        print('Done.\n')
        
        print('Calculate emissivity...')
        self.calc_e()
        print('Done.\n')
        
    def add_substrate(self):
        '''
        Add substrate
        '''
        
        if self.substrate == 'land':
            print(f'Substrate: {self.substrate}')
            print(f'Roughness RMS: {self.roughness_rms}')
            self.substrate = make_soil('soil_wegmuller',
                                  permittivity_model=complex(10, 1),
                                  roughness_rms=self.roughness_rms,
                                  temperature=self.surface_temp)

        if self.substrate == 'MYI':
            print(f'Substrate: {self.substrate}')
            # prepare inputs
            l = 9  # 9 ice layers
            thickness = np.array([1.5 / l] * l)  # ice is 1.5m thick
            p_ex = np.array([1.0e-3] * (l))  # correlation length
            temperature = np.linspace(self.surface_temp, 273.15 - 1.8,
                                      l)  # temperature gradient in the ice from T deg K at top to freezing temperature of water at bottom (-1.8 deg C)
            salinity = np.linspace(2., 10.,
                                   l) * PSU  # salinity profile ranging from salinity=2 at the top to salinity=10 at the bottom of the ice

            # create a multi-year sea ice column with assumption of spherical brine inclusions (brine_inclusion_shape="spheres"), and 10% porosity:
            ice_type = 'multiyear'  # first-year or multi-year sea ice
            porosity = 0.08  # ice porosity in fractions, [0..1]

            self.substrate = make_ice_column(ice_type=ice_type, thickness=thickness,
                                        temperature=self.surface_temp,
                                        microstructure_model="exponential",
                                        brine_inclusion_shape="spheres",
                                        # brine_inclusion_shape can be "spheres", "random_needles" or "mix_spheres_needles"
                                        salinity=salinity,
                                        # either 'salinity' or 'brine_volume_fraction' should be given for sea ice; if salinity is given, brine volume fraction is calculated in the model; if none is given, ice is treated as fresh water ice
                                        porosity=porosity,
                                        # either density or 'porosity' should be set for sea ice. If porosity is given, density is calculated in the model. If none is given, ice is treated as having a porosity of 0% (no air inclusions)
                                        corr_length=p_ex,
                                        add_water_substrate="ocean"  # see comment below
                                        )
        
        # 1K atmosphere
        self.Tbdown = 1
        self.atmosphere1K = SimpleIsotropicAtmosphere(tbdown=self.Tbdown,
                                                      tbup=0,
                                                      trans=1)

        # Add snowpack on top of substrate:
        self.medium = self.snowpack + self.substrate
   
    def calc_e(self):
        '''
        Calculate Emissivity
        '''

        # Run model without atmsophere
        sresult_0 = self.m.run(self.radiometer, self.medium)
        # Run model without 1K atmosphere
        self.medium.atmosphere = self.atmosphere1K
        sresult_1 = self.m.run(self.radiometer, self.medium)

        # V-pol
        if self.e_equation == 1:
            print(f'Equation {self.e_equation}')
            self.emissivity_V = 1 - (sresult_1.TbV() - sresult_0.TbV()) / self.Tbdown
        elif self.e_equation == 2:
            print(f'Equation {self.e_equation}')
            self.emissivity_V = (sresult_0.TbV()) / self.surface_temp
        else:
            raise('Please specify correct equation number (1/2)')
        print(f'Ev={self.emissivity_V}')

        # H-pol
        if self.e_equation == 1:
            print(f'Equation {self.e_equation}')
            #reflectivity_H = ( sresult_1.TbH() - sresult_0.TbH() ) / Tbdown
            self.emissivity_H = 1 - (sresult_1.TbH() - sresult_0.TbH()) / self.Tbdown
        elif self.e_equation == 2:
            print(f'Equation {self.e_equation}')
            self.emissivity_H = (sresult_0.TbH()) / self.surface_temp
        else:
            raise('Please specify correct equation number (1/2)')
        print(f'Eh={self.emissivity_H}')
        
        # From old code (incorrect?)
        #reflectivity_H = (sresult_0.TbH() + sresult_1.TbH()) / 2 / T

In [103]:
e = Emissivity()

Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [320]
Snow temperature: 265
Sphere radius: 0.0003
Stickiness: 0.2
Done.

Adding sustrate...
Substrate: land
Roughness RMS: 0.01
Done.

Calculate emissivity...
Equation 1
Ev=0.9094322948010074
Equation 1
Eh=0.8350242135876726
Done.



In [ ]:
##############################
# Calculate emissivity
##############################

sn_ms_model_list = ['autocorrelation', 'exponential', 'gaussian_random_field',
                            'homogeneous', 'independent_sphere', 'sampled_autocorrelation',
                            'sticky_hard_spheres','test_autocorrelation', 'test_exponential',
                            'test_sticky_hard_spheres', 'teubner_strey', 'unified_autocorrelation',
                            'unified_scaled_exponential', 'unified_sticky_hard_spheres', 'unified_teubner_strey']

densities_list = list(range(100, 700, 100))
fq_list = [7e9, 11e9, 19e9, 24e9, 37e9, 89e9]
theta = 55
polarizations = ['h','v']
sn_th_max = 3.
sn_th_min = 0.01
sn_th_step = 0.1
substrate = 'MYI'
roughness_rms = 0.01
sn_ms_model = 6
stickiness=0.2
radius=0.3e-3
radius_um = int(radius*10**6)

out_path = '/home/denis/git/smrt_experiments/sea_ice/myi'

for stickiness in np.arange(0.1, 0.4, 0.1):
    for radius in np.arange(0.1e-3, 0.45e-3, 0.05e-3):
        radius_um = int(radius*10**6)
        d_res = {}

        for sn_density in densities_list:
            d_res[sn_density]= {}
            for pol in polarizations:
                d_res[sn_density][pol]= {}
                d_res[sn_density][pol]= {}

        ll_e1_h, ll_e1_v = [], []

        for sn_density in densities_list:
            print(f'Snowpack density: {sn_density}')
            for idx, fq in enumerate(fq_list):
                print(f'Frequency: {fq/1e9}')
                for s_th in np.arange(sn_th_min, sn_th_max, sn_th_step):
                    print(f'Snow thickness: {s_th}')

                    # Snow and model parameters
                    model_parameters = {}
                    model_parameters['sn_thickness'] = s_th
                    model_parameters['sn_density'] = sn_density
                    model_parameters['e_equation'] = 1
                    model_parameters['substrate'] = substrate
                    model_parameters['roughness_rms'] = roughness_rms
                    model_parameters['stickiness'] = stickiness
                    model_parameters['radius'] = radius
                    model_parameters['sn_ms_model'] = sn_ms_model
                    model_parameters['sn_temp'] = 265
                    model_parameters['surface_temp'] =265

                    # Instrumental parameters
                    instr_parameters = {}
                    instr_parameters['fq_list'] = fq
                    instr_parameters['polarizations'] = polarizations
                    instr_parameters['theta'] = theta

                    e = Emissivity(model_parameters=model_parameters,
                                   instr_parameters=instr_parameters)    

                    ll_e1_h.append(e.emissivity_H)
                    ll_e1_v.append(e.emissivity_V)

                fq_str = fq / 1e9
                d_res[sn_density]['h'][fq_str] = {}
                d_res[sn_density]['v'][fq_str] = {}
                d_res[sn_density]['h'][fq_str]['e1'] = ll_e1_h
                d_res[sn_density]['v'][fq_str]['e1'] = ll_e1_v
                ll_e1_h, ll_e1_v = [], []

        ########################
        # Plot results
        ########################

        ncols = 3
        nrows = int(np.ceil(len(d_res.keys()) / ncols))

        if substrate == 'land':
            substrate_title = substrate + ' (Wegmuller)'
        else:
            substrate_title = substrate

        plt.clf()
        fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 12))

        r_ch = 0
        c_ch = 0
        colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]

        for i, density in enumerate(d_res.keys()):
            if r_ch >= ncols:
                r_ch = 0
                c_ch += 1
            pol = 'h'
            for i_color, ikey in enumerate(d_res[density][pol]):
                ax[c_ch, r_ch].plot(np.arange(sn_th_min, sn_th_max, sn_th_step), d_res[density][pol][ikey]['e1'],
                                    label=f'{pol.upper()} {ikey} GHz',
                                    linestyle='dashed', c=colors[i_color])
            text = f'Sn. density={density}$\ kg/m^3,\ \Theta$={theta}$^\circ$' \
                f'\nSubstrate:$\ $ {substrate_title}' \
                f'\nSurface roughnes={roughness_rms}' \
                f'\nMicr. model={sn_ms_model_list[sn_ms_model]}' \
                f'\nStickiness={stickiness}' \
                f'\nCrystal R={radius_um}$\ \mu m$'
            ax[c_ch, r_ch].text(.01, .01, text, ha='left', va='bottom',
                                transform=ax[c_ch, r_ch].transAxes,
                                fontsize='medium')
            pol = 'v'
            for i_color, ikey in enumerate(d_res[density][pol]):
                ax[c_ch, r_ch].plot(np.arange(sn_th_min, sn_th_max, sn_th_step), d_res[density][pol][ikey]['e1'],
                                    label=f'{pol.upper()} {ikey} GHz',
                                    linewidth=3, c=colors[i_color])
            ax[c_ch, r_ch].legend(loc='lower right', prop={'size': 8})
            ax[c_ch, r_ch].grid(linewidth=0.15)
            ax[c_ch, r_ch].set_ylim([0.25, 1.01])
            if r_ch > 0:
                ax[c_ch, r_ch].axes.yaxis.set_ticklabels([])
            if r_ch == 0:
                ax[c_ch, r_ch].set_ylabel('Emissivity')
            r_ch += 1

        plt.xlabel('Snow thickness, m')
        plt.subplots_adjust(hspace=0.1)
        plt.subplots_adjust(wspace=0.1)
        fig.patch.set_facecolor('white')
        plt.savefig(f'{out_path}/E_{substrate}_r{radius_um}_s{stickiness}_{sn_ms_model}.png', bbox_inches='tight', dpi=300)

Snowpack density: 100
Frequency: 7.0
Snow thickness: 0.01
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9837742775996503
Equation 1
Eh=0.8398412544167968
Done.

Snow thickness: 0.11
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.983779857495847
Equation 1
Eh=0.8400039477796781
Done.

Snow thickness: 0.21000000000000002
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9837854311674619
Equation 1
Eh=0.8401664

Equation 1
Ev=0.9839119589593679
Equation 1
Eh=0.8438514782250763
Done.

Snow thickness: 2.61
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9839173896455122
Equation 1
Eh=0.8440094402643865
Done.

Snow thickness: 2.71
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9839228145967809
Equation 1
Eh=0.8441672163137639
Done.

Snow thickness: 2.81
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9839282338282942
E

Equation 1
Ev=0.9743777972510657
Equation 1
Eh=0.8383669568575272
Done.

Snow thickness: 2.21
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9743725289757208
Equation 1
Eh=0.8387274010740668
Done.

Snow thickness: 2.31
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.974367274960855
Equation 1
Eh=0.8390867568056706
Done.

Snow thickness: 2.41
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9743620356629208
Eq

Equation 1
Ev=0.9610282999954052
Equation 1
Eh=0.8379361131996177
Done.

Snow thickness: 1.81
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9607239059979804
Equation 1
Eh=0.8387103581628139
Done.

Snow thickness: 1.9100000000000001
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9604222946838377
Equation 1
Eh=0.8394758796684982
Done.

Snow thickness: 2.01
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9601

Equation 1
Ev=0.9537375782815047
Equation 1
Eh=0.8369772400369584
Done.

Snow thickness: 1.4100000000000001
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9528426605145057
Equation 1
Eh=0.8378257146387966
Done.

Snow thickness: 1.51
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9519613409326837
Equation 1
Eh=0.838654258920883
Done.

Snow thickness: 1.61
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [100]
Snow temperature: 265
Sphere radius: 0.0001
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.95109

...

Equation 1
Ev=0.9319987256574223
Equation 1
Eh=0.8142251497225743
Done.

Snow thickness: 0.91
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [500]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9319986251359182
Equation 1
Eh=0.8142250450998745
Done.

Snow thickness: 1.01
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [500]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9319986046786823
Equation 1
Eh=0.8142250236305415
Done.

Snow thickness: 1.11
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [500]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.931998600495546

Equation 1
Ev=0.9895327377187186
Equation 1
Eh=0.8444326204166828
Done.

Snow thickness: 0.41000000000000003
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9896075109204503
Equation 1
Eh=0.8445470339195253
Done.

Snow thickness: 0.51
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.98968168568598
Equation 1
Eh=0.8446605581170274
Done.

Snow thickness: 0.6100000000000001
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equa

Equation 1
Ev=0.9912990139790736
Equation 1
Eh=0.8471402622002415
Done.

Frequency: 11.0
Snow thickness: 0.01
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9800726931969734
Equation 1
Eh=0.8346908207187198
Done.

Snow thickness: 0.11
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9804215786488726
Equation 1
Eh=0.8351363001076209
Done.

Snow thickness: 0.21000000000000002
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...


Equation 1
Ev=0.9867619173319326
Equation 1
Eh=0.8432994306947137
Done.

Snow thickness: 2.51
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9869805507817659
Equation 1
Eh=0.8435820367463123
Done.

Snow thickness: 2.61
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9871952632447574
Equation 1
Eh=0.8438596209910827
Done.

Snow thickness: 2.71
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.987406133405045

Equation 1
Ev=0.9888157665791368
Equation 1
Eh=0.8469675899215474
Done.

Snow thickness: 2.01
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9893054828155528
Equation 1
Eh=0.8475573179792661
Done.

Snow thickness: 2.11
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.9897705890885504
Equation 1
Eh=0.8481174481516405
Done.

Snow thickness: 2.21
Start initializing snowpack with:
Microstructure model: sticky_hard_spheres
Snow density: [600]
Snow temperature: 265
Sphere radius: 0.00025
Stickiness: 0.1
Done.

Adding sustrate...
Substrate: MYI
Done.

Calculate emissivity...
Equation 1
Ev=0.990212375730436